In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np

import sklearn
import sklearn.feature_extraction.text as sklearnText
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


from functools import partial

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *


import re
import string
import difflib


C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
import pandas as pd
def multiplier(x):
    return x * 100

In [2]:
data_set = pd.read_csv("Data/fuzz.csv")

In [3]:
def stringmatcher(dataframe, column1, column2):
    return difflib.SequenceMatcher(None, str(dataframe[column1]), str(dataframe[column2])).ratio()

In [4]:
# Adds the sequence matching ratio to the dataframe
data_set['SMratio'] = data_set.apply(partial(stringmatcher, column1='question1', column2='question2'), axis=1)

In [28]:
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,bog1,bog2
0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,...,92.105263,step step guide invest share market india,step step guide invest share market,92.105263,100.000000,step step guide invest share market india!SPLI...,92.030662,89.553242,"{'step': 2, 'guide': 1, 'invest': 1, 'share': ...","{'step': 2, 'guide': 1, 'invest': 1, 'share': ..."
1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,...,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673,42.105263,story kohinoor kohinoor diamond!SPLIT!would ha...,42.425131,46.117093,"{'story': 1, 'kohinoor': 2, 'diamond': 1}","{'would': 1, 'happen': 1, 'indian': 1, 'govern..."
2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,...,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000,30.000000,increase speed internet connection using vpn!S...,22.576485,22.576485,"{'increase': 1, 'speed': 1, 'internet': 1, 'co...","{'internet': 1, 'speed': 1, 'increased': 1, 'h..."
3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,...,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820,6.666667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000,"{'mentally': 1, 'lonely': 1, 'solve': 1}","{'find': 1, 'remainder': 1, 'math2324math': 1,..."
4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,...,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101,26.315789,one dissolve water quikly sugar salt methane c...,27.205959,16.836842,"{'one': 1, 'dissolve': 1, 'water': 1, 'quikly'...","{'fish': 1, 'would': 1, 'survive': 1, 'salt': ..."


In [93]:
def split_train_test(X, y, split_random_state = 1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = split_random_state)
    return X_train, X_test, y_train, y_test

def create_classifier(hidden_layer_sizes_tuple, max_iterations):
    '''create mlp classifiers with parameters'''
    from sklearn.neural_network import MLPClassifier       
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes_tuple, max_iter=max_iterations)
    return mlp
    
def create_regressor(hidden_layer_sizes_tuple, max_iterations):
    from sklearn.neural_network import MLPRegressor
    '''create mlp classifiers with parameters'''
    mlp = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes_tuple, max_iter=max_iterations)
    return mlp

def createNeuralNetwork(X_train, X_test, y_train, y_test, hidden_layer_tuples, max_iterations, kind = 'classification', 
                        split_random_state = 1, use_Scaler = True):
    ''' create neural network and evaluate using training and test set.
        Standard scaler by default enabled, 'use_Scaler = False' to disable.
        'hidden_layer_tuples' & 'max_iterations' to define the parameters of the neural network
    '''
    # if enabled, use a standard scaler on the data
    if use_Scaler == True:
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()

        # Fit the training data
        scaler.fit(X_train)

        StandardScaler(copy=True, with_mean=True, with_std=True)

        # Apply the transformations to the data:
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
    if kind == 'classification':
        # create the classifier
        mlp = create_classifier(hidden_layer_tuples, max_iterations)
        mlp.fit(X_train,y_train)
        predictions = mlp.predict(X_test)
        from sklearn.metrics import classification_report,confusion_matrix
        try:
            print(confusion_matrix(y_test,predictions))
        except:
            print()
        
    elif kind == 'regression':
        mlp = create_regressor(hidden_layer_tuples, max_iterations)
        mlp.fit(X_train,y_train)
        predictions = mlp.predict(X_test)
    
    # run only if y is know, so do not run if classifying unknown data 
    if y_test!= []:
        score_train = mlp.score(X_train, y_train)
        score_test = mlp.score(X_test, y_test)
        print(score_train, score_test)
    
    #print(predictions)
    return predictions
    
#X_train, X_test, y_train, y_test = split_train_test(data_set.iloc[:, -4:], data_set['is_duplicate_y'], split_random_state = 1)
#createNeuralNetwork(X_train, X_test, y_train, y_test, (30,20,15,15), 3000, kind = 'classification')

# Running on the test set (a.k.a. the assignment)

### create FuzzyWuzzy paramaters as input

In [6]:
test_data = pd.read_csv("Data/test_data.csv")

In [7]:
test_data['merged'] = test_data['question1'] + '!SPLIT!' + test_data['question2']

In [9]:
def calcFuzzRation(merged):
    merged = str(merged)
    merged = merged.split('!SPLIT!')
    try:
        fuzzRatio = fuzz.ratio(merged[0],merged[1])
    except:
        fuzzRatio = 0
    
    return fuzzRatio

def calcFuzzPartial(merged):
    merged = str(merged)
    merged = merged.split('!SPLIT!')
    try:
        fuzzRatio = fuzz.partial_ratio(merged[0],merged[1])
    except:
        fuzzRatio = 0
    
    return fuzzRatio

def calcFuzzTokenSort(merged):
    merged = str(merged)
    merged = merged.split('!SPLIT!')
    try:
        fuzzRatio = fuzz.token_sort_ratio(merged[0],merged[1])
    except:
        fuzzRatio = 0
    
    return fuzzRatio

def calcFuzzTokenSet(merged):
    merged = str(merged)
    merged = merged.split('!SPLIT!')
    try:
        fuzzRatio = fuzz.token_set_ratio(merged[0],merged[1])
    except:
        fuzzRatio = 0
    
    return fuzzRatio

In [10]:
test_data['fuzzRatio'] = test_data['merged'].apply(calcFuzzRation)

In [11]:
test_data['fuzzPartial'] = test_data['merged'].apply(calcFuzzPartial)

In [12]:
test_data['fuzzTokenSort'] = test_data['merged'].apply(calcFuzzTokenSort)

In [13]:
test_data['fuzzTokenSet'] = test_data['merged'].apply(calcFuzzTokenSet)

In [14]:
test_data['SMratio'] = test_data.apply(partial(stringmatcher, column1='question1', column2='question2'), axis=1)

In [15]:
test_data.head()

,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio
0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,0.510870
1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,0.637681
2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,0.447059
3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,0.074074
4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,0.876712


In [15]:
# Hiermee haalt hij momenteel de 70% accuracy
X_train = data_set.iloc[:, -5:]
y_train = data_set['is_duplicate_y']

# there is no training data available, we only need to classify
y_test = []
X_test = test_data.iloc[:, -5:]



In [47]:
output = createNeuralNetwork(X_train, X_test, y_train, y_test, (30,20,15,15), 3000, kind = 'classification')

In [48]:
len(X_test),len(output)

(81126, 81126)

In [49]:
len(test_data)

81126

In [52]:
from collections import Counter
Counter(output)

Counter({0: 47762, 1: 33364})

In [53]:
submission = pd.concat([test_data.iloc[:, 0], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,1
2,21,1
3,23,0
4,34,0


In [54]:
submission.to_csv('result/submission.csv', index = False)

# 2
Ingebouwde bag of words, gekloot vooral

In [80]:
lol = ["this is stupid", "lol test"]
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(lol)
print(format(vect.vocabulary_))

{'this': 4, 'is': 0, 'stupid': 2, 'lol': 1, 'test': 3}


In [81]:
data_set.question1.values.astype('U')

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., 'What is one coin?',
       'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?',
       'What is like to have sex with cousin?'], dtype='<U623')

In [82]:
# Dit maakt dus zo'n rare sparsematrix. Het heeft ook geen zin om er
# een array van te maken aangezien de lengte daarvan een stuk langer is. 
# Het is hierdoor vrij lastig om deze door dat neural network heen te gooien.
vector = CountVectorizer().fit(data_set.merged.values.astype('U'))
Q1 = vect.transform(data_set.merged.values.astype('U'))
#print(format(repr(X_train)))

In [86]:
Q1

<323164x5 sparse matrix of type '<class 'numpy.int64'>'
	with 159352 stored elements in Compressed Sparse Row format>

In [85]:
len(Q1.toarray())

323164

In [61]:
# Test met merged questions als input
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(), Q1, y_train, cv=5)
print(format(np.mean(scores)))

0.6311965445413981


In [63]:
from sklearn.model_selection import GridSearchCV
param = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param, cv=5)
grid.fit(Q1, y_train)
print(format(grid.best_score_))
print(grid.best_params_)

0.6311965441695239
{'C': 0.1}


In [68]:
y = grid.predict(testQ1)

In [82]:
# Werkt dus om een of andere reden echt totaal niet..
from collections import Counter
Counter(y)

Counter({0: 81118, 1: 8})

In [83]:
testQ1 = vect.transform(test_data.merged.values.astype('U'))

# 3

In [12]:
# Basis functie die even alle onnodige worden eruit haalt

def cleantext(text):
    text = str(text)
    stripped = re.sub("!SPLIT!"," ",text)
    stripped = re.sub(r'[^\w\s]','',stripped)
    stripped = stripped.lower()
    stopwordss = set(stopwords.words('english'))
    tokens = word_tokenize(stripped)
    stripped = [word for word in tokens if word not in stopwordss]
    stemmer = nltk.PorterStemmer()
    result = [stemmer.stem(word) for word in stripped]
    result = ' '.join(stripped)
    
    return result

In [17]:
data_set['clean1'] = data_set.question1.apply(cleantext)

In [19]:
data_set['clean2'] = data_set.question2.apply(cleantext)

In [20]:
test_data['clean1'] = test_data.question1.apply(cleantext)

In [22]:
test_data['clean2'] = test_data.question2.apply(cleantext)

In [23]:
data_set['SMratioClean'] = data_set.apply(partial(stringmatcher, column1='clean1', column2='clean2'), axis=1)

In [24]:
data_set['SMratio'] = data_set.apply(partial(stringmatcher, column1='clean1', column2='clean2'), axis=1)

In [25]:
test_data['SMratioClean'] = test_data.apply(partial(stringmatcher, column1='clean1', column2='clean2'), axis=1)

In [26]:
test_data['SMratio'] = test_data.apply(partial(stringmatcher, column1='clean1', column2='clean2'), axis=1)

In [27]:
data_set.head()

,Unnamed: 0,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean
0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,98,93,100,0.921053,step step guide invest share market india,step step guide invest share market,0.921053
1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,73,63,86,0.591837,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0.591837
2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,41,63,63,0.550000,increase speed internet connection using vpn,internet speed increased hacking dns,0.550000
3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,20,24,28,0.229508,mentally lonely solve,find remainder math2324math divided 2423,0.229508
4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,54,47,67,0.247191,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0.247191


In [7]:
import pandas as pd
def multiplier(x):
    return x * 100

In [29]:
df = pd.DataFrame([[4,9],] * 3, columns = ['a', 'b'])
df['a'] = df['a'].apply(multiplier)
df

,a,b
0,400,9
1,400,9
2,400,9


In [30]:
data_set['SMratio'] = data_set['SMratio'].apply(multiplier)
data_set['SMratioClean'] = data_set['SMratioClean'].apply(multiplier)
test_data['SMratio'] = test_data['SMratio'].apply(multiplier)
test_data['SMratioClean'] = test_data['SMratioClean'].apply(multiplier)

In [31]:
test_data.head()

,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean
0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,48.175182,would trump presidency mean current internatio...,trump presidency affect students presently us ...,48.175182
1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,66.666667,rockets look white,rockets boosters painted white,66.666667
2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,52.000000,whats causing someone jealous,avoid jealous someone,52.000000
3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,25.000000,much 30 kv hp,find conversion chart cc horsepower,25.000000
4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,86.363636,best travel website spain,best travel website,86.363636


In [32]:
data_set.head()

,Unnamed: 0,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean
0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,98,93,100,92.105263,step step guide invest share market india,step step guide invest share market,92.105263
1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,73,63,86,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673
2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,41,63,63,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000
3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,20,24,28,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820
4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,54,47,67,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101


In [33]:
test_data.to_csv('data/testset.csv', index = False)

In [34]:
data_set.to_csv('data/datasettrain.csv', index = False)

In [69]:
# Hiermee haalt hij momenteel de 70% accuracy
X_train = data_set[['fuzzRatio', 'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratioClean']]
y_train = data_set['is_duplicate_y']

# there is no training data available, we only need to classify
y_test = []
X_test = test_data[['fuzzRatio', 'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratioClean']]


In [70]:
output = createNeuralNetwork(X_train, X_test, y_train, y_test, (30,20,15,15), 3000, kind = 'classification')

In [71]:
submission = pd.concat([test_data.iloc[:, 0], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,1
2,21,0
3,23,0
4,34,0


In [72]:
submission.to_csv('result/submission.csv', index = False)

In [7]:
# https://www.linkedin.com/pulse/text-classification-using-bag-words-approach-nltk-scikit-rajendran

# 4

In [4]:
test_data = pd.read_csv("Data/testset.csv")
test_data.head()

,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean
0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,48.175182,would trump presidency mean current internatio...,trump presidency affect students presently us ...,48.175182
1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,66.666667,rockets look white,rockets boosters painted white,66.666667
2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,52.000000,whats causing someone jealous,avoid jealous someone,52.000000
3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,25.000000,much 30 kv hp,find conversion chart cc horsepower,25.000000
4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,86.363636,best travel website spain,best travel website,86.363636


In [5]:
data_set = pd.read_csv("Data/datasettrain.csv")
data_set.head()

,Unnamed: 0,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean
0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,98,93,100,92.105263,step step guide invest share market india,step step guide invest share market,92.105263
1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,73,63,86,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673
2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,41,63,63,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000
3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,20,24,28,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820
4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,54,47,67,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101


In [6]:
(len(data_set), len(test_data))

(323164, 81126)

In [58]:
from nltk.stem.lancaster import LancasterStemmer
def jaccard_sim(dataframe, question1, question2):
    question1 = word_tokenize(str(dataframe[question1]))
    question2 = word_tokenize((str(dataframe[question2])))
    lemma = nltk.LancasterStemmer()
    question1= [lemma.stem(word) for word in question1]
    question2= [lemma.stem(word) for word in question2]
    intersecti = [word for word in question1 if word in question2]
    return float(len(intersecti)) / (len(question1) + len(question2) - len(intersecti))

In [ ]:
def stringmatcher(dataframe, column1, column2):
    return difflib.SequenceMatcher(None, str(dataframe[column1]), str(dataframe[column2])).ratio()

In [54]:
jaccard_sim('This is great', 'This isnt great')

0.5

In [59]:
data_set['jaccard'] = data_set.apply(partial(jaccard_sim, question1='question1', question2='question2'), axis=1)

In [60]:
test_data['jaccard'] = test_data.apply(partial(jaccard_sim, question1='question1', question2='question2'), axis=1)

In [62]:
# Hiermee haalt hij momenteel de 72% accuracy
X_train = data_set[['fuzzRatio', 'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratioClean', 'jaccard']]
y_train = data_set['is_duplicate_y']

y_test = []
X_test = test_data[['fuzzRatio', 'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratioClean', 'jaccard']]


In [63]:
output = createNeuralNetwork(X_train, X_test, y_train, y_test, (30,20,15,15), 3000, kind = 'classification')

In [64]:
submission = pd.concat([test_data.iloc[:, 0], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,1
2,21,0
3,23,0
4,34,0


In [65]:
submission.to_csv('result/submission.csv', index = False)

# Testen andere classifiers

In [2]:
test_data = pd.read_csv("Data/testset2.csv")
test_data.head()

,Unnamed: 0,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean
0,0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,48.175182,would trump presidency mean current internatio...,trump presidency affect students presently us ...,48.175182,0.161290,would trump presidency mean current internatio...,0.099746,0.168657
1,1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,66.666667,rockets look white,rockets boosters painted white,66.666667,0.400000,rockets look white!SPLIT!rockets boosters pain...,0.344642,0.411207
2,2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,52.000000,whats causing someone jealous,avoid jealous someone,52.000000,0.357143,whats causing someone jealous!SPLIT!avoid jeal...,0.380817,0.411207
3,3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,25.000000,much 30 kv hp,find conversion chart cc horsepower,25.000000,0.052632,much 30 kv hp!SPLIT!find conversion chart cc h...,0.000000,0.000000
4,4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,86.363636,best travel website spain,best travel website,86.363636,0.777778,best travel website spain!SPLIT!best travel we...,0.776515,0.776515


In [3]:
data_set = pd.read_csv("Data/datasettrain2.csv")
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean
0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,...,93,100,92.105263,step step guide invest share market india,step step guide invest share market,92.105263,1.000000,step step guide invest share market india!SPLI...,0.920307,0.895532
1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,...,63,86,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673,0.421053,story kohinoor kohinoor diamond!SPLIT!would ha...,0.424251,0.461171
2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,...,63,63,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000,0.300000,increase speed internet connection using vpn!S...,0.225765,0.225765
3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,...,24,28,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820,0.066667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000
4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,...,47,67,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101,0.263158,one dissolve water quikly sugar salt methane c...,0.272060,0.168368


In [6]:
data_set['jaccard'] = data_set['jaccard'].apply(multiplier)
data_set['cosine'] = data_set['cosine'].apply(multiplier)
data_set['cosineClean'] = data_set['cosineClean'].apply(multiplier)

In [7]:
test_data['jaccard'] = test_data['jaccard'].apply(multiplier)
test_data['cosine'] = test_data['cosine'].apply(multiplier)
test_data['cosineClean'] = test_data['cosineClean'].apply(multiplier)

In [22]:

def create_bog(dataframe, string1, string2):
    q = list(set(word_tokenize(str(dataframe[string1]))))
    merged = list(set(word_tokenize(str(dataframe[string2]))))
    onehot = str()
    for word in merged:
        if word in q and word != '!' and word != 'SPLIT':
            onehot+='1'
        elif word not in q and word != '!' and word != 'SPLIT':
            onehot+='0'
    return int(onehot)

In [19]:
test = data_set.loc[[10]]


test['onehot1'] = test.apply(partial(create_bog, string1='clean1', string2='cleanMerged'), axis=1)
test['onehot2'] = test.apply(partial(create_bog, string1='clean2', string2='cleanMerged'), axis=1)
test

14
14


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,onehot1,onehot2
10,10,10,10,10,Method to find separation of slits using fresn...,What are some of the things technicians can te...,NaN,0,Method to find separation of slits using fresn...,21,...,27.826087,method find separation slits using fresnel bip...,things technicians tell durability reliability...,27.826087,7.142857,method find separation slits using fresnel bip...,7.234714,0.0,11000001110110,111110001001


In [23]:
data_set['onehot1'] = data_set.apply(partial(create_bog, string1='clean1', string2='cleanMerged'), axis=1)
data_set['onehot2'] = data_set.apply(partial(create_bog, string1='clean2', string2='cleanMerged'), axis=1)
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,onehot1,onehot2
0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,...,92.105263,step step guide invest share market india,step step guide invest share market,92.105263,100.000000,step step guide invest share market india!SPLI...,92.030662,89.553242,111111,111110
1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,...,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673,42.105263,story kohinoor kohinoor diamond!SPLIT!would ha...,42.425131,46.117093,11000010,101111111
2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,...,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000,30.000000,increase speed internet connection using vpn!S...,22.576485,22.576485,101101011,110010101
3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,...,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820,6.666667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000,1000110,10111001
4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,...,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101,26.315789,one dissolve water quikly sugar salt methane c...,27.205959,16.836842,1111110001111,1011110000


In [24]:
test_data['onehot1'] = test_data.apply(partial(create_bog, string1='clean1', string2='cleanMerged'), axis=1)
test_data['onehot2'] = test_data.apply(partial(create_bog, string1='clean2', string2='cleanMerged'), axis=1)
test_data.head()

,Unnamed: 0,test_id,question1,question2,merged,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,onehot1,onehot2
0,0,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,What would a Trump presidency mean for current...,53,54,56,61,48.175182,would trump presidency mean current internatio...,trump presidency affect students presently us ...,48.175182,16.129032,would trump presidency mean current internatio...,9.974596,16.865689,111001110101011,111011010101
1,1,20,Why do rockets look white?,Why are rockets and boosters painted white?,Why do rockets look white?!SPLIT!Why are rocke...,64,65,66,81,66.666667,rockets look white,rockets boosters painted white,66.666667,40.000000,rockets look white!SPLIT!rockets boosters pain...,34.464214,41.120705,10110,1111
2,2,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,What's causing someone to be jealous?!SPLIT!Wh...,59,65,75,78,52.000000,whats causing someone jealous,avoid jealous someone,52.000000,35.714286,whats causing someone jealous!SPLIT!avoid jeal...,38.081653,41.120705,10111,1011
3,3,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,How much is 30 kV in HP?!SPLIT!Where can I fin...,25,33,30,30,25.000000,much 30 kv hp,find conversion chart cc horsepower,25.000000,5.263158,much 30 kv hp!SPLIT!find conversion chart cc h...,0.000000,0.000000,101100100,10011011
4,4,34,What is the best travel website in spain?,What is the best travel website?,What is the best travel website in spain?!SPLI...,88,97,87,100,86.363636,best travel website spain,best travel website,86.363636,77.777778,best travel website spain!SPLIT!best travel we...,77.651453,77.651453,1111,1110


In [29]:
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,SMratio,clean1,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,bog1,bog2
0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,...,92.105263,step step guide invest share market india,step step guide invest share market,92.105263,100.000000,step step guide invest share market india!SPLI...,92.030662,89.553242,"{'step': 2, 'guide': 1, 'invest': 1, 'share': ...","{'step': 2, 'guide': 1, 'invest': 1, 'share': ..."
1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,...,59.183673,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,59.183673,42.105263,story kohinoor kohinoor diamond!SPLIT!would ha...,42.425131,46.117093,"{'story': 1, 'kohinoor': 2, 'diamond': 1}","{'would': 1, 'happen': 1, 'indian': 1, 'govern..."
2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,...,55.000000,increase speed internet connection using vpn,internet speed increased hacking dns,55.000000,30.000000,increase speed internet connection using vpn!S...,22.576485,22.576485,"{'increase': 1, 'speed': 1, 'internet': 1, 'co...","{'internet': 1, 'speed': 1, 'increased': 1, 'h..."
3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,...,22.950820,mentally lonely solve,find remainder math2324math divided 2423,22.950820,6.666667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000,"{'mentally': 1, 'lonely': 1, 'solve': 1}","{'find': 1, 'remainder': 1, 'math2324math': 1,..."
4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,...,24.719101,one dissolve water quikly sugar salt methane c...,fish would survive salt water,24.719101,26.315789,one dissolve water quikly sugar salt methane c...,27.205959,16.836842,"{'one': 1, 'dissolve': 1, 'water': 1, 'quikly'...","{'fish': 1, 'would': 1, 'survive': 1, 'salt': ..."


In [76]:
def tdidfmean(merged):
    try:
        print(hoi)
        merged = str(dataframe[merged])
        merged = merged.split('!SPLIT!')
        corpus = [merged[0],merged[1]]
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(corpus)
        print(X)
       # if q == 'question1':
        v1 = np.array([x  for y in X[0].toarray() for x in y])
        print(v1)
        return np.mean(v1)
        #if q == 2:
        #    v2 = np.array([x  for y in X[1].toarray() for x in y])
        #    return np.mean(v2)
    except:
        return 0

In [77]:
data_set['tfidf1'] = data_set['cleanMerged'].apply(tdidfmean)

In [72]:
data_set.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,question1,question2,is_duplicate_x,is_duplicate_y,merged,fuzzRatio,...,clean2,SMratioClean,jaccard,cleanMerged,cosine,cosineClean,onehot1,onehot2,tdidf1,tfidf1
0,0,0,0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,NaN,0,What is the step by step guide to invest in sh...,93,...,step step guide invest share market,92.105263,100.000000,step step guide invest share market india!SPLI...,92.030662,89.553242,111111,111110,0,0
1,1,1,1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,NaN,0,What is the story of Kohinoor (Koh-i-Noor) Dia...,65,...,would happen indian government stole kohinoor ...,59.183673,42.105263,story kohinoor kohinoor diamond!SPLIT!would ha...,42.425131,46.117093,11000010,101111111,0,0
2,2,2,2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,NaN,0,How can I increase the speed of my internet co...,45,...,internet speed increased hacking dns,55.000000,30.000000,increase speed internet connection using vpn!S...,22.576485,22.576485,101101011,110010101,0,0
3,3,3,3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,NaN,0,Why am I mentally very lonely? How can I solve...,7,...,find remainder math2324math divided 2423,22.950820,6.666667,mentally lonely solve!SPLIT!find remainder mat...,0.000000,0.000000,1000110,10111001,0,0
4,4,4,4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,NaN,0,"Which one dissolve in water quikly sugar, salt...",37,...,fish would survive salt water,24.719101,26.315789,one dissolve water quikly sugar salt methane c...,27.205959,16.836842,1111110001111,1011110000,0,0


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(max_features=500)


In [14]:
features = vectorizer.fit_transform(data_set.merged.values.astype('U'))

In [15]:
X = vectorizer.transform(data_set.merged.values.astype('U'))

In [16]:
X

<317205x500 sparse matrix of type '<class 'numpy.int64'>'
	with 3618898 stored elements in Compressed Sparse Row format>

In [17]:
names = vectorizer.get_feature_names()
len(names)

500

In [18]:
names[0]

'10'

In [19]:
dense = X.todense()

In [20]:
count_vect_df = pd.DataFrame(X.todense(), columns=names)

In [21]:
count_vect_df.shape

(317205, 500)

In [9]:
data_set2 = data_set

In [11]:
data_set2 = data_set2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'id', 'question1',
       'question2', 'is_duplicate_x', 'is_duplicate_y', 'merged','cleanMerged',
         'clean1', 'clean2'], axis=1)
data_set2.head()

,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,SMratioClean,jaccard,cosine,cosineClean
0,93,98,93,100,92.105263,92.105263,100.000000,92.030662,89.553242
1,65,73,63,86,59.183673,59.183673,42.105263,42.425131,46.117093
2,45,41,63,63,55.000000,55.000000,30.000000,22.576485,22.576485
3,7,20,24,28,22.950820,22.950820,6.666667,0.000000,0.000000
4,37,54,47,67,24.719101,24.719101,26.315789,27.205959,16.836842


In [28]:
data_set.shape

(317205, 21)

In [23]:
new = pd.concat([data_set2, count_vect_df], axis=1)
new.head()

,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,SMratioClean,jaccard,cosine,cosineClean,10,...,worth,would,write,writing,wrong,year,years,you,your,youtube
0,93,98,93,100,92.105263,92.105263,100.000000,92.030662,89.553242,0,...,0,0,0,0,0,0,0,0,0,0
1,65,73,63,86,59.183673,59.183673,42.105263,42.425131,46.117093,0,...,0,1,0,0,0,0,0,0,0,0
2,45,41,63,63,55.000000,55.000000,30.000000,22.576485,22.576485,0,...,0,0,0,0,0,0,0,0,0,0
3,7,20,24,28,22.950820,22.950820,6.666667,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
4,37,54,47,67,24.719101,24.719101,26.315789,27.205959,16.836842,0,...,0,1,0,0,0,0,0,0,0,0


In [24]:
# TEst
X = vectorizer.transform(test_data.merged.values.astype('U'))

In [25]:
dense = X.todense()

In [26]:
count_vect_df = pd.DataFrame(X.todense(), columns=names)

In [27]:
test_data.columns

Index(['Unnamed: 0', 'test_id', 'question1', 'question2', 'merged',
       'fuzzRatio', 'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratio',
       'clean1', 'clean2', 'SMratioClean', 'jaccard', 'cleanMerged', 'cosine',
       'cosineClean'],
      dtype='object')

In [29]:
test_data2 = test_data.drop(['Unnamed: 0', 'test_id', 'question1',
       'question2', 'merged','cleanMerged',
         'clean1', 'clean2'], axis=1)
test_data2.head()

,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,SMratioClean,jaccard,cosine,cosineClean
0,53,54,56,61,48.175182,48.175182,16.129032,9.974596,16.865689
1,64,65,66,81,66.666667,66.666667,40.000000,34.464214,41.120705
2,59,65,75,78,52.000000,52.000000,35.714286,38.081653,41.120705
3,25,33,30,30,25.000000,25.000000,5.263158,0.000000,0.000000
4,88,97,87,100,86.363636,86.363636,77.777778,77.651453,77.651453


In [30]:
newtest = pd.concat([test_data2, count_vect_df], axis=1)
newtest.head()

,fuzzRatio,fuzzPartial,fuzzTokenSort,fuzzTokenSet,SMratio,SMratioClean,jaccard,cosine,cosineClean,10,...,worth,would,write,writing,wrong,year,years,you,your,youtube
0,53,54,56,61,48.175182,48.175182,16.129032,9.974596,16.865689,0,...,0,1,0,0,0,0,0,0,0,0
1,64,65,66,81,66.666667,66.666667,40.000000,34.464214,41.120705,0,...,0,0,0,0,0,0,0,0,0,0
2,59,65,75,78,52.000000,52.000000,35.714286,38.081653,41.120705,0,...,0,0,0,0,0,0,0,0,0,0
3,25,33,30,30,25.000000,25.000000,5.263158,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
4,88,97,87,100,86.363636,86.363636,77.777778,77.651453,77.651453,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X_train = new #data_set[['cosine', 'cosineClean', 'fuzzRatio','fuzzPartial','fuzzTokenSort',
                    #'fuzzTokenSet', 'SMratio','SMratioClean', 'jaccard']]
y_train = data_set[['is_duplicate_y']]
X_test = newtest#test_data[['cosine', 'cosineClean', 'fuzzRatio','fuzzPartial','fuzzTokenSort',
                    #'fuzzTokenSet', 'SMratio','SMratioClean', 'jaccard']]
y_test = []
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [46]:
X_train.columns

Index(['cosine', 'cosineClean', 'fuzzRatio', 'fuzzPartial', 'fuzzTokenSort',
       'fuzzTokenSet', 'SMratio', 'SMratioClean', 'jaccard'],
      dtype='object')

In [47]:
data_set.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'id', 'question1',
       'question2', 'is_duplicate_x', 'is_duplicate_y', 'merged', 'fuzzRatio',
       'fuzzPartial', 'fuzzTokenSort', 'fuzzTokenSet', 'SMratio', 'clean1',
       'clean2', 'SMratioClean', 'jaccard', 'cleanMerged', 'cosine',
       'cosineClean'],
      dtype='object')

In [94]:
output = createNeuralNetwork(X_train, X_test, y_train, y_test, (45,30,10,15), 10000, kind = 'classification')

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [95]:
Counter(output)

Counter({0: 46841, 1: 34285})

In [106]:
submission = pd.concat([test_data.iloc[:, 1], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,1
2,21,0
3,23,0
4,34,0


In [107]:
submission.to_csv('result/submission.csv', index = False)

In [112]:
from sklearn.linear_model import LogisticRegression
lr =  LogisticRegression()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_train)

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [35]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_train))

0.909030437729544


In [115]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred=nb.predict(X_train)

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [116]:
print(accuracy_score(y_pred, y_train))

0.6796361974117684


In [117]:
from sklearn.linear_model import SGDClassifier
sgd =  SGDClassifier(loss='modified_huber', shuffle=True, random_state=101)
sgd.fit(X_train, y_train)
y_pred=sgd.predict(X_train)
print(accuracy_score(y_pred, y_train))

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6307845084409136


In [89]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 20)
knn.fit(X_train, y_train)
y_pred=knn.predict(X_train)
print(accuracy_score(y_pred, y_train))

C:\Users\Marit\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.7612931700319983


In [91]:
output=knn.predict(X_test)
print(output)
Counter(output)

[0 1 1 ... 0 0 0]


Counter({0: 53139, 1: 27987})

In [123]:
submission = pd.concat([test_data.iloc[:, 1], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,1
2,21,1
3,23,0
4,34,1


In [125]:
submission.to_csv('result/submission.csv', index = False)

In [121]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=10, random_state=101, max_features=None, min_samples_leaf = 15)
tree.fit(X_train, y_train)
y_pred=tree.predict(X_train)
print(accuracy_score(y_pred, y_train))

0.7282672089027601


In [33]:
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(n_estimators=50, oob_score=True, n_jobs=-1, random_state=None, max_features=None, min_samples_leaf=10)
rfm.fit(X_train, y_train)
y_pred=rfm.predict(X_train)


C:\Users\Marit\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'accuracy_score' is not defined

In [34]:
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [ ]:
print(accuracy_score(y_pred, y_train))

In [85]:
print(rfm.feature_importances_)

[0.10446326 0.39754382 0.11239477 0.05998104 0.05675085 0.06269446
 0.06755248 0.06416946 0.07444985]


In [36]:
output=rfm.predict(X_test)
print(output)
Counter(output)

[0 0 0 ... 0 0 0]


Counter({0: 50912, 1: 30214})

In [37]:
submission = pd.concat([test_data.iloc[:, 1], pd.DataFrame(output)], axis=1, sort=False)
submission.head()

,test_id,0
0,15,0
1,20,0
2,21,0
3,23,0
4,34,0


In [38]:
submission.to_csv('result/submission.csv', index = False)

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel="linear", C=0.025, random_state=101)
svm.fit(X_train, y_train)
y_pred=svm.predict(X_train)
print(accuracy_score(y_pred, y_train))

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
d = {'col1': [1, 2], 'col2': [[3,2], [3,4]]}

from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier(n_estimators=100, oob_score=True, n_jobs=-1, random_state=10, max_features=None, min_samples_leaf=10)
rfm.fit(d[['col2']], d[['col1']])
y_pred=rfm.predict(d[['col2']])
print(accuracy_score(y_pred, d[['col1']]))

TypeError: unhashable type: 'list'

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),])


In [35]:
text_clf.fit(data_set.merged.values.astype('U'),data_set[['is_duplicate_y']])

C:\Users\Marit\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [38]:
predicted = text_clf.predict(data_set.merged.values.astype('U'))
print(accuracy_score(predicted, data_set[['is_duplicate_y']]))

0.7546003373212906


In [39]:
ypred = text_clf.predict(test_data.merged.values.astype('U'))

In [40]:
from collections import Counter
Counter(ypred)

Counter({1: 17662, 0: 63464})